In [14]:
from collections import Counter
import numpy as np
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
import torch.nn as nn

#from torch_rnn_classifier import TorchRNNClassifier
#from torch_tree_nn import TorchTreeNN
#import sst
#import utils

In [18]:
data = pd.read_csv("aita_processed.csv")
data

,Unnamed: 0,id,timestamp,title,body,edited,verdict,score,num_comments,is_asshole,text,age,gender
0,0,1ytxov,1.393279e+09,[AITA] I wrote an explanation in TIL and came ...,[Here is the post in question](http://www.redd...,False,asshole,52,13.0,1,[AITA] I wrote an explanation in TIL and came ...,NaN,NaN
1,1,1yu29c,1.393281e+09,[AITA] Threw my parent's donuts away,"My parents are diabetic, morbidly obese, and a...",1393290576.0,asshole,140,27.0,1,[AITA] Threw my parent's donuts away My parent...,NaN,NaN
2,2,1yu8hi,1.393285e+09,I told a goth girl she looked like a clown.,I was four.,False,not the asshole,74,15.0,0,I told a goth girl she looked like a clown. I ...,NaN,NaN
3,3,1yuc78,1.393287e+09,[AItA]: Argument I had with another redditor i...,http://www.reddit.com/r/HIMYM/comments/1vvfkq/...,1393286962.0,everyone sucks,22,3.0,1,[AItA]: Argument I had with another redditor i...,NaN,NaN
4,4,1yueqb,1.393288e+09,[AITA] I let my story get a little long and bo...,NaN,False,not the asshole,6,4.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97623,97623,ex94w5,1.580577e+09,AITA for telling my sister she is being a spoi...,My sister(17F) and I(15M) are white kids born ...,1580585457.0,not the asshole,16,23.0,0,AITA for telling my sister she is being a spoi...,15.0,M
97624,97624,ex970f,1.580577e+09,AITA for telling my husband to f* off after he...,My husband (28M) and I (32F) are married for a...,1580584475.0,not the asshole,1373,304.0,0,AITA for telling my husband to f* off after he...,32.0,F
97625,97625,ex9dwo,1.580578e+09,AITA for attempting to keep my students out of...,Upfront apologies for formatting. I’m also try...,False,not the asshole,4,15.0,0,AITA for attempting to keep my students out of...,NaN,NaN
97626,97626,ex9egs,1.580578e+09,WIBTA if I left my brothers fate up to the state?,A little back story my mom is a drug addict an...,False,not the asshole,280,140.0,0,WIBTA if I left my brothers fate up to the sta...,NaN,NaN


In [25]:
print(str(len(data)))

97628


In [24]:
print (data['verdict'].value_counts())

not the asshole     59114
asshole             20959
no assholes here    12003
everyone sucks       5552
Name: verdict, dtype: int64


In [3]:
print(data["age"].notnull().sum())
print(data["gender"].notnull().sum())

13957
9862


In [4]:
count = 0
for index,row in data.iterrows():
    if pd.notnull(data.at[index,"age"]):
        if pd.notnull(data.at[index,"gender"]):
            count += 1
print(count)

9340


## Sample selections to test gender and age extraction

There are a total of 97628 samples. We were able to extract 13957 age and 9862 gender data points. 9340 samples had both the age and gender extracted (9.7% of the total dataset, 67.6% of all age data, 95.6% of all gender data).  An additional 4617 samples had only age data, and 522 samples had only gender data. 

We will create 3 sets of 100 samples to validate the age and gender extraction. Each set will consist of 50 samples with both age and gender data, 15 samples with only gender but no age data, 15 samples with age but no gender data, and 20 samples with no age or gender data. Though the amount of data points with age and/or gender data is less than 10% of the entire dataset, we are oversampling from this subsection of the data because we are more adverse to false positives than false negatives. 

In [9]:
with_age = data[data["age"].notnull()]
with_gender = data[data["gender"].notnull()]
no_age = data[data["age"].isnull()]
with_age_and_gender = with_age[with_age["gender"].notnull()]
with_age_no_gender = with_age[with_age["gender"].isnull()]
with_gender_no_age = with_gender[with_gender["age"].isnull()]
no_age_no_gender = no_age[no_age["gender"].isnull()]

In [10]:
sampled_age_and_gender = with_age_and_gender.sample(n=150)
sampled_age_no_gender = with_age_no_gender.sample(n=45)
sampled_gender_no_age = with_gender_no_age.sample(n=45)
sampled_no_age_no_gender = no_age_no_gender.sample(n=60)
sample1_ag, sample2_ag, sample3_ag = np.array_split(sampled_age_and_gender,3)
sample1_a, sample2_a, sample3_a = np.array_split(sampled_age_no_gender,3)
sample1_g, sample2_g, sample3_g = np.array_split(sampled_gender_no_age,3)
sample1_, sample2_, sample3_ = np.array_split(sampled_no_age_no_gender,3)
sample1 = pd.concat([sample1_ag, sample1_a, sample1_g, sample1_])
sample2 = pd.concat([sample2_ag, sample2_a, sample2_g, sample2_])
sample3 = pd.concat([sample3_ag, sample3_a, sample3_g, sample3_])

In [11]:
sample1.to_csv("sample1_age_and_gender_extraction.csv")
sample2.to_csv("sample2_age_and_gender_extraction.csv")
sample3.to_csv("sample3_age_and_gender_extraction.csv")

In [83]:
print(sampled_age_no_gender.head())

       Unnamed: 0      id     timestamp  \
16004       16004  av9w3i  1.551247e+09   
3194         3194  8s41f1  1.529364e+09   
76314       76314  dv6tny  1.573545e+09   
64212       64212  d72xpt  1.569022e+09   
90460       90460  ekon0h  1.578284e+09   

                                                   title  \
16004  AITA for ending a potential relationship becau...   
3194   AITA for wanting to kick my sister out? And no...   
76314                          WIBTA if I went to the RA   
64212  WIBTA if I told my sister it's best not to be ...   
90460  AITA for asking my boyfriend to get rid of his...   

                                                    body        edited  \
16004  I'm (17F) a senior in high school this year, a...         False   
3194   Backstory. Me (29) sister (25) after going thr...         False   
76314  Throwaway account, on mobile.\n\nI (19) curren...         False   
64212   I'll try and keep this as short and together ...  1569022756.0   
90460  Bac